In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
from configs import spark_config as config
from utils import spark_utils as utils

spark = utils.create_spark_session("merge_deliveries",{
    'spark.executor.memory': '5g',
    'spark.executor.cores': '6',
})

matches = spark.read.csv(os.path.join(config.PROCESSED_DATA_DIR, 'matches.csv'), inferSchema=True, header=True)
deliveries = spark.read.csv(os.path.join(config.PROCESSED_DATA_DIR, 'deliveries.csv'), inferSchema=True, header=True)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-24T14:55:08.646+0530] {spark_utils.py:17} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/24 14:55:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/24 14:55:12 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


[2024-11-24T14:55:13.185+0530] {spark_utils.py:37} INFO - Spark session created successfully.


In [2]:
matches = matches.drop('date','city','toss_winner','toss_decision')
matches.show(5)

+--------+-----------+----------+------+-------+-----------+
|match_id|      team1|     team2|gender| season|     winner|
+--------+-----------+----------+------+-------+-----------+
| 1019981|New Zealand|Bangladesh|  male|2016/17|New Zealand|
| 1031665|West Indies|   England|  male|   2017|West Indies|
| 1001349|  Australia| Sri Lanka|  male|2016/17|  Sri Lanka|
| 1034825|      India|   England|  male|2016/17|    England|
| 1019979|New Zealand|Bangladesh|  male|2016/17|New Zealand|
+--------+-----------+----------+------+-------+-----------+
only showing top 5 rows



In [3]:
deliveries = deliveries.drop('season','start_date','venue','striker','non_striker','bowler')
deliveries.show(5)

+--------+-------+----+------------+------------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+
|match_id|innings|ball|batting_team|bowling_team|runs_off_bat|extras|wides|noballs|byes|legbyes|penalty|wicket_type|player_dismissed|other_wicket_type|other_player_dismissed|
+--------+-------+----+------------+------------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+
| 1306389|      1| 0.1|     Bahrain|Saudi Arabia|           0|     0|    0|      0|   0|      0|      0|          0|               0|                0|                     0|
| 1306389|      1| 0.2|     Bahrain|Saudi Arabia|           0|     1|    1|      0|   0|      0|      0|          0|               0|                0|                     0|
| 1306389|      1| 0.3|     Bahrain|Saudi Arabia|           0|     1|    1|      0|   0|      0|      0|          0|         

In [4]:
from pyspark.sql import Window
from pyspark.sql.functions import coalesce, col, lit, sum as F_sum

# Calculate "runs" as the row-wise sum of specified columns
deliveries = deliveries.withColumn(
    "runs",
    coalesce(col("runs_off_bat"), lit(0)) +
    coalesce(col("extras"), lit(0)) +
    coalesce(col("penalty"), lit(0))
)

# Drop the original columns that were summed
deliveries = deliveries.drop("runs_off_bat", "extras", "wides", "noballs", "byes", "legbyes", "penalty")

# Calculate "wickets" as the row-wise sum of dismissals, handling null values
deliveries = deliveries.withColumn(
    "wickets",
    (coalesce(col("player_dismissed").cast("int"), lit(0)) +
     coalesce(col("other_player_dismissed").cast("int"), lit(0)))
)

# Drop columns related to wicket types and dismissed players that are no longer needed
deliveries = deliveries.drop("wicket_type", "player_dismissed", "other_wicket_type", "other_player_dismissed")
deliveries.show(5)

+--------+-------+----+------------+------------+----+-------+
|match_id|innings|ball|batting_team|bowling_team|runs|wickets|
+--------+-------+----+------------+------------+----+-------+
| 1306389|      1| 0.1|     Bahrain|Saudi Arabia|   0|      0|
| 1306389|      1| 0.2|     Bahrain|Saudi Arabia|   1|      0|
| 1306389|      1| 0.3|     Bahrain|Saudi Arabia|   1|      0|
| 1306389|      1| 0.4|     Bahrain|Saudi Arabia|   2|      0|
| 1306389|      1| 0.5|     Bahrain|Saudi Arabia|   0|      0|
+--------+-------+----+------------+------------+----+-------+
only showing top 5 rows



In [5]:
#Define the window specifications for cumulative sums partitioned by "match_id" and "innings"
window_spec = Window.partitionBy("match_id", "innings").orderBy("ball")

# Calculate cumulative sum for "runs" as "curr_score"
deliveries = deliveries.withColumn(
    "curr_score",
    F_sum("runs").over(window_spec)
)

# Calculate cumulative sum for "wickets" as "curr_wickets"
deliveries = deliveries.withColumn(
    "curr_wickets",
    F_sum("wickets").over(window_spec)
)

# Calculate the required columns for the final DataFrame


# Display the resulting DataFrame
deliveries.show(250)

+--------+-------+----+------------+------------+----+-------+----------+------------+
|match_id|innings|ball|batting_team|bowling_team|runs|wickets|curr_score|curr_wickets|
+--------+-------+----+------------+------------+----+-------+----------+------------+
|  211048|      2| 0.1| New Zealand|   Australia|   0|      0|         0|           0|
|  211048|      2| 0.2| New Zealand|   Australia|   0|      0|         0|           0|
|  211048|      2| 0.3| New Zealand|   Australia|   1|      0|         1|           0|
|  211048|      2| 0.4| New Zealand|   Australia|   0|      0|         1|           0|
|  211048|      2| 0.5| New Zealand|   Australia|   1|      0|         2|           0|
|  211048|      2| 0.6| New Zealand|   Australia|   0|      0|         2|           0|
|  211048|      2| 1.1| New Zealand|   Australia|   4|      0|         6|           0|
|  211048|      2| 1.2| New Zealand|   Australia|   1|      0|         7|           0|
|  211048|      2| 1.3| New Zealand|   Aust

In [6]:
data=deliveries.join(matches,on='match_id').drop('season','venue','gender','team1','team2')
data.sort('match_id').show(10)

+--------+-------+----+------------+------------+----+-------+----------+------------+-------+
|match_id|innings|ball|batting_team|bowling_team|runs|wickets|curr_score|curr_wickets| winner|
+--------+-------+----+------------+------------+----+-------+----------+------------+-------+
|  211028|      1| 0.1|     England|   Australia|   0|      0|         0|           0|England|
|  211028|      1| 0.2|     England|   Australia|   1|      0|         1|           0|England|
|  211028|      1| 0.3|     England|   Australia|   0|      0|         1|           0|England|
|  211028|      1| 0.4|     England|   Australia|   0|      0|         1|           0|England|
|  211028|      1| 0.5|     England|   Australia|   0|      0|         1|           0|England|
|  211028|      1| 0.6|     England|   Australia|   1|      0|         2|           0|England|
|  211028|      1| 0.7|     England|   Australia|   2|      0|         4|           0|England|
|  211028|      1| 1.1|     England|   Australia| 

In [7]:
from pyspark.sql import functions as F
window_spec = Window.partitionBy("match_id").orderBy("innings","ball")

data = data.withColumn(
    "won",
    F.when(((data["batting_team"] == data["winner"]) & (data['innings']==2)) | 
            ((data["bowling_team"] == data["winner"]) & (data['innings']==1))
    , 1).otherwise(0)
).drop("batting_team","bowling_team","winner").sort("match_id","innings","ball")
data.show(240)

+--------+-------+----+----+-------+----------+------------+---+
|match_id|innings|ball|runs|wickets|curr_score|curr_wickets|won|
+--------+-------+----+----+-------+----------+------------+---+
|  211028|      1| 0.1|   0|      0|         0|           0|  0|
|  211028|      1| 0.2|   1|      0|         1|           0|  0|
|  211028|      1| 0.3|   0|      0|         1|           0|  0|
|  211028|      1| 0.4|   0|      0|         1|           0|  0|
|  211028|      1| 0.5|   0|      0|         1|           0|  0|
|  211028|      1| 0.6|   1|      0|         2|           0|  0|
|  211028|      1| 0.7|   2|      0|         4|           0|  0|
|  211028|      1| 1.1|   0|      0|         4|           0|  0|
|  211028|      1| 1.2|   0|      0|         4|           0|  0|
|  211028|      1| 1.3|   1|      0|         5|           0|  0|
|  211028|      1| 1.4|   0|      0|         5|           0|  0|
|  211028|      1| 1.5|   0|      0|         5|           0|  0|
|  211028|      1| 1.6|  

In [8]:
window_spec = Window.partitionBy("match_id").orderBy( "innings", "ball")
window_spec_ffill = Window.partitionBy("match_id").orderBy( "innings", "ball").rowsBetween(Window.unboundedPreceding, 0)

# Calculate the max of "curr_score" in 1st innings as "target" otherwise forward fill
data = data.withColumn(
    "target",
    F.when(
        (F.col("innings") == 1) & (F.col("curr_score") == F.max("curr_score").over(window_spec)),
        F.col("curr_score")
    ).otherwise(F.lit(None))
)
data = data.withColumn("overs", col("ball").cast("int"))
data = data.withColumn("run_rate",
                       F.when(F.col("overs")!=0,
                       col("curr_score")/col("overs")
                       ).otherwise(0).cast("float"))

# Forward fill the "target" column
data = data.withColumn("target", F.last("target", ignorenulls=True).over(window_spec_ffill))
data = data.withColumn("target", F.when(col("innings") == 1, 0).otherwise(col("target")+1)).orderBy(col("match_id"),col("innings"), col("ball"))

data.show(240)

+--------+-------+----+----+-------+----------+------------+---+------+-----+---------+
|match_id|innings|ball|runs|wickets|curr_score|curr_wickets|won|target|overs| run_rate|
+--------+-------+----+----+-------+----------+------------+---+------+-----+---------+
|  211028|      1| 0.1|   0|      0|         0|           0|  0|     0|    0|      0.0|
|  211028|      1| 0.2|   1|      0|         1|           0|  0|     0|    0|      0.0|
|  211028|      1| 0.3|   0|      0|         1|           0|  0|     0|    0|      0.0|
|  211028|      1| 0.4|   0|      0|         1|           0|  0|     0|    0|      0.0|
|  211028|      1| 0.5|   0|      0|         1|           0|  0|     0|    0|      0.0|
|  211028|      1| 0.6|   1|      0|         2|           0|  0|     0|    0|      0.0|
|  211028|      1| 0.7|   2|      0|         4|           0|  0|     0|    0|      0.0|
|  211028|      1| 1.1|   0|      0|         4|           0|  0|     0|    1|      4.0|
|  211028|      1| 1.2|   0|    

In [9]:
data = data.withColumn("required_run_rate",
                F.when(col("innings")==1,0).otherwise((col("target") - col("curr_score"))/(20-col("overs"))).cast("float"))
data = data.select("match_id","innings","ball","runs","wickets","curr_score","curr_wickets","overs","run_rate","required_run_rate","target","won")
data.count()
data.show(240)

+--------+-------+----+----+-------+----------+------------+-----+---------+-----------------+------+---+
|match_id|innings|ball|runs|wickets|curr_score|curr_wickets|overs| run_rate|required_run_rate|target|won|
+--------+-------+----+----+-------+----------+------------+-----+---------+-----------------+------+---+
|  211028|      1| 0.1|   0|      0|         0|           0|    0|      0.0|              0.0|     0|  0|
|  211028|      1| 0.2|   1|      0|         1|           0|    0|      0.0|              0.0|     0|  0|
|  211028|      1| 0.3|   0|      0|         1|           0|    0|      0.0|              0.0|     0|  0|
|  211028|      1| 0.4|   0|      0|         1|           0|    0|      0.0|              0.0|     0|  0|
|  211028|      1| 0.5|   0|      0|         1|           0|    0|      0.0|              0.0|     0|  0|
|  211028|      1| 0.6|   1|      0|         2|           0|    0|      0.0|              0.0|     0|  0|
|  211028|      1| 0.7|   2|      0|         4

In [10]:
sampled_match_id = data.filter(data["won"]==1).select("match_id").distinct().sample(False, 0.1).limit(1).collect()[0]["match_id"]
data.filter((data["match_id"] == sampled_match_id)).show(271)

+--------+-------+----+----+-------+----------+------------+-----+---------+-----------------+------+---+
|match_id|innings|ball|runs|wickets|curr_score|curr_wickets|overs| run_rate|required_run_rate|target|won|
+--------+-------+----+----+-------+----------+------------+-----+---------+-----------------+------+---+
| 1257948|      1| 0.1|   1|      0|         1|           0|    0|      0.0|              0.0|     0|  1|
| 1257948|      1| 0.2|   1|      0|         2|           0|    0|      0.0|              0.0|     0|  1|
| 1257948|      1| 0.3|   1|      0|         3|           0|    0|      0.0|              0.0|     0|  1|
| 1257948|      1| 0.4|   4|      0|         7|           0|    0|      0.0|              0.0|     0|  1|
| 1257948|      1| 0.5|   1|      0|         8|           0|    0|      0.0|              0.0|     0|  1|
| 1257948|      1| 0.6|   1|      0|         9|           0|    0|      0.0|              0.0|     0|  1|
| 1257948|      1| 0.7|   0|      0|         9

In [11]:
utils.spark_save_data(data,config.MERGED_DATA_DIR, "ball_by_ball.csv")
spark.stop()

[2024-11-24T14:55:44.769+0530] {spark_utils.py:64} INFO - Successfully wrote data to /usr/ravi/t20/data/3_mergedData/ball_by_ball.csv
